# Getting Started - Basic XCT Analysis

This notebook provides an interactive introduction to the XCT Thermomagnetic Analysis Framework. Learn how to:

- **Load XCT volumes** from multiple file formats
- **Segment volumes** using different methods
- **Compute core metrics** (volume, surface area, void fraction)
- **Visualize results** interactively
- **Use the XCTAnalyzer class** for streamlined analysis

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Load XCT data from various formats (DICOM, TIFF, RAW, CSV, etc.)
2. Perform segmentation using Otsu thresholding
3. Compute and interpret basic morphological metrics
4. Use interactive widgets for parameter exploration
5. Visualize 3D volumes and 2D slices

## ⚠️ Prerequisites

Before using this notebook, ensure:
1. **Jupyter Widgets Extension** is enabled:
   ```bash
   jupyter nbextension enable --py widgetsnbextension --user
   # or for JupyterLab:
   jupyter labextension install @jupyter-widgets/jupyterlab-manager
   ```
2. **Required packages** are installed:
   ```bash
   pip install -r ../requirements.txt
   pip install ipywidgets pyvista matplotlib pandas numpy seaborn
   ```
3. **XCT Data** is available in `../data/` directory or accessible file paths

## 📖 Usage

1. Run all cells to initialize the widgets
2. Load an XCT volume from file
3. Adjust segmentation parameters and preview results
4. Compute metrics and view results
5. Explore visualizations interactively


## 1. Setup and Imports

Import required libraries and set up the environment.


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for PyVista (for 3D visualization)
try:
    import pyvista as pv
    PYVISTA_AVAILABLE = True
    pv.set_jupyter_backend('static')
except ImportError:
    PYVISTA_AVAILABLE = False
    print("⚠️ PyVista not available. 3D visualization will be limited.")
    print("   Install with: pip install pyvista")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path for imports
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Getting Started - Basic XCT Analysis")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")
print(f"   PyVista available: {PYVISTA_AVAILABLE}")
print("   Loading modules...")


📦 Getting Started - Basic XCT Analysis
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True
   PyVista available: True
   Loading modules...


## 2. Load Framework Modules

Import the XCT analysis modules.


In [ ]:
# Load XCT analysis modules
try:
    from src.analyzer import XCTAnalyzer
    from src.core.segmentation import segment_volume, otsu_threshold
    from src.core.metrics import compute_all_metrics
    from src.core.visualization import visualize_3d_volume, visualize_slice
    from src.utils.utils import load_volume, convert_to_mm, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise
except Exception as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Data Explorer

Use the interactive widgets below to load, segment, and analyze XCT volumes.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    current_volume = None
    analysis_results = {}
    
    # ============================================
    # Section 1: File Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path (e.g., ../data/sample.dcm)',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'MHD/MHA', 'NumPy', 'CSV', 'Excel'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    # For RAW files
    dimensions_x = widgets.IntText(value=512, description='Width:', style={'description_width': 'initial'}, disabled=True)
    dimensions_y = widgets.IntText(value=512, description='Height:', style={'description_width': 'initial'}, disabled=True)
    dimensions_z = widgets.IntText(value=512, description='Depth:', style={'description_width': 'initial'}, disabled=True)
    dtype_dropdown = widgets.Dropdown(
        options=['uint8', 'uint16', 'float32', 'float64'],
        value='uint16',
        description='Data Type:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    # Voxel size
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    voxel_size_unit = widgets.Dropdown(
        options=['mm', 'um', 'cm'],
        value='mm',
        description='Unit:',
        style={'description_width': 'initial'}
    )
    
    normalize_check = widgets.Checkbox(
        value=True,
        description='Normalize volume',
        indent=False
    )
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='120px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Segmentation
    # ============================================
    
    segmentation_method = widgets.Dropdown(
        options=['Otsu', 'Multi-threshold', 'Adaptive', 'Manual'],
        value='Otsu',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    manual_threshold = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.01,
        description='Threshold:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    refine_segmentation_check = widgets.Checkbox(
        value=True,
        description='Apply morphological refinement',
        indent=False
    )
    
    min_object_size = widgets.IntText(
        value=100,
        description='Min Object Size:',
        style={'description_width': 'initial'}
    )
    
    fill_holes_check = widgets.Checkbox(
        value=True,
        description='Fill holes',
        indent=False
    )
    
    segment_button = widgets.Button(
        description='🎯 Segment',
        button_style='success',
        layout=widgets.Layout(width='150px')
    )
    
    segmentation_preview = Output(layout=widgets.Layout(height='350px'))
    
    # ============================================
    # Section 3: Metrics Display
    # ============================================
    
    compute_metrics_button = widgets.Button(
        description='📊 Compute Metrics',
        button_style='info',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    metrics_display = widgets.HTML(
        value="<p><i>No metrics computed</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    # ============================================
    # Section 4: Visualization
    # ============================================
    
    slice_axis = widgets.Dropdown(
        options=['X (depth)', 'Y (height)', 'Z (width)'],
        value='Z (width)',
        description='Slice Axis:',
        style={'description_width': 'initial'}
    )
    
    slice_index = widgets.IntSlider(
        value=0,
        min=0,
        max=100,
        step=1,
        description='Slice Index:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    visualize_button = widgets.Button(
        description='🎨 Visualize',
        button_style='warning',
        layout=widgets.Layout(width='150px')
    )
    
    visualization_output = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready to analyze</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


In [4]:
if WIDGETS_AVAILABLE:
    
    def update_segmentation_method(change):
        """Enable/disable manual threshold based on method"""
        if change['new'] == 'Manual':
            manual_threshold.disabled = False
        else:
            manual_threshold.disabled = True
    
    def update_raw_params(change):
        """Show/hide RAW parameters"""
        if change['new'] == 'RAW':
            dimensions_x.disabled = False
            dimensions_y.disabled = False
            dimensions_z.disabled = False
            dtype_dropdown.disabled = False
        else:
            dimensions_x.disabled = True
            dimensions_y.disabled = True
            dimensions_z.disabled = True
            dtype_dropdown.disabled = True
    
    def load_volume_callback(button):
        """Load XCT volume from file"""
        global analyzer, current_volume
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        file_path_obj = Path(file_path)
        if not file_path_obj.exists():
            # Try relative to data directory
            data_path = project_root / 'data' / file_path
            if data_path.exists():
                file_path_obj = data_path
            else:
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            # Get voxel size
            voxel_size = (
                float(voxel_size_x.value),
                float(voxel_size_y.value),
                float(voxel_size_z.value)
            )
            
            # Convert to mm if needed
            if voxel_size_unit.value != 'mm':
                voxel_size = convert_to_mm(voxel_size, voxel_size_unit.value)
            
            # Initialize analyzer
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            # Load volume
            if file_format_dropdown.value == 'RAW':
                dimensions = (dimensions_x.value, dimensions_y.value, dimensions_z.value)
                dtype_map = {'uint8': np.uint8, 'uint16': np.uint16, 
                           'float32': np.float32, 'float64': np.float64}
                dtype = dtype_map[dtype_dropdown.value]
                analyzer.load_volume(
                    str(file_path_obj),
                    dimensions=dimensions,
                    dtype=dtype,
                    normalize=normalize_check.value
                )
            else:
                analyzer.load_volume(str(file_path_obj), normalize=normalize_check.value)
            
            progress_bar.value = 80
            current_volume = analyzer.volume
            
            # Update slice index range
            axis_map = {'X (depth)': 0, 'Y (height)': 1, 'Z (width)': 2}
            axis_idx = axis_map[slice_axis.value]
            slice_index.max = analyzer.volume.shape[axis_idx] - 1
            slice_index.value = analyzer.volume.shape[axis_idx] // 2
            slice_index.disabled = False
            
            # Update volume info
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Data Type:</b> {analyzer.volume.dtype}</p>
            <p><b>Voxel Size:</b> {analyzer.voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            <p><b>Intensity Range:</b> [{analyzer.volume.min():.2f}, {analyzer.volume.max():.2f}]</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def segment_volume_callback(button):
        """Segment the loaded volume"""
        global analyzer
        
        if analyzer is None or analyzer.volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Segmenting volume...</p>"
        progress_bar.value = 20
        
        try:
            method_map = {
                'Otsu': 'otsu',
                'Multi-threshold': 'multi',
                'Adaptive': 'adaptive',
                'Manual': 'manual'
            }
            method = method_map[segmentation_method.value]
            
            kwargs = {}
            if method == 'manual':
                kwargs['threshold'] = manual_threshold.value
            
            # Segment
            segmented = analyzer.segment(
                method=method,
                refine=refine_segmentation_check.value,
                min_object_size=min_object_size.value,
                fill_holes_flag=fill_holes_check.value,
                **kwargs
            )
            
            progress_bar.value = 80
            
            # Preview segmentation
            with segmentation_preview:
                clear_output()
                middle_slice_idx = segmented.shape[0] // 2
                fig, axes = plt.subplots(1, 2, figsize=(12, 5))
                
                # Original slice
                original_slice = analyzer.volume[middle_slice_idx, :, :]
                axes[0].imshow(original_slice, cmap='gray')
                axes[0].set_title(f'Original (Slice {middle_slice_idx})')
                axes[0].axis('off')
                
                # Segmented slice
                segmented_slice = segmented[middle_slice_idx, :, :]
                axes[1].imshow(segmented_slice, cmap='gray')
                axes[1].set_title(f'Segmented (Slice {middle_slice_idx})')
                axes[1].axis('off')
                
                plt.tight_layout()
                plt.show()
                
                material_fraction = np.mean(segmented > 0)
                print(f"Material Fraction: {material_fraction:.2%}")
                print(f"Void Fraction: {1-material_fraction:.2%}")
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Segmentation complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error segmenting: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def compute_metrics_callback(button):
        """Compute and display metrics"""
        global analyzer, analysis_results
        
        if analyzer is None or analyzer.segmented_volume is None:
            status_display.value = "<p style='color: red;'>Please load and segment a volume first</p>"
            return
        
        status_display.value = "<p>Computing metrics...</p>"
        progress_bar.value = 20
        
        try:
            # Compute metrics
            metrics = analyzer.compute_metrics(include_surface_area=True)
            analysis_results['metrics'] = metrics
            progress_bar.value = 80
            
            # Display metrics
            metrics_html = f"""
            <h4>📏 Core Metrics</h4>
            <table style='width:100%; border-collapse: collapse;'>
            <tr style='background-color: #f0f0f0;'>
                <th style='padding: 8px; text-align: left; border: 1px solid #ddd;'>Metric</th>
                <th style='padding: 8px; text-align: right; border: 1px solid #ddd;'>Value</th>
            </tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Volume</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('volume', 0):.3f} mm³</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Surface Area</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('surface_area', 0):.3f} mm²</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Void Fraction</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('void_fraction', 0):.2%}</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Relative Density</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('relative_density', 0):.2%}</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Specific Surface Area</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('specific_surface_area', 0):.3f} mm²/mm³</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Material Voxels</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('material_voxels', 0):,}</td></tr>
            <tr><td style='padding: 8px; border: 1px solid #ddd;'>Void Voxels</td>
                <td style='padding: 8px; text-align: right; border: 1px solid #ddd;'>{metrics.get('void_voxels', 0):,}</td></tr>
            </table>
            """
            metrics_display.value = metrics_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Metrics computed successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error computing metrics: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def update_slice_range(change):
        """Update slice index range when axis changes"""
        if analyzer and analyzer.volume is not None:
            axis_map = {'X (depth)': 0, 'Y (height)': 1, 'Z (width)': 2}
            axis_idx = axis_map[change['new']]
            slice_index.max = analyzer.volume.shape[axis_idx] - 1
            slice_index.value = analyzer.volume.shape[axis_idx] // 2
    
    def visualize_callback(button):
        """Visualize volume"""
        global analyzer
        
        if analyzer is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Generating visualization...</p>"
        progress_bar.value = 20
        
        try:
            with visualization_output:
                clear_output()
                
                # Determine which volume to show
                volume_to_show = analyzer.segmented_volume if analyzer.segmented_volume is not None else analyzer.volume
                title = "Segmented Volume" if analyzer.segmented_volume is not None else "Original Volume"
                
                # Get slice
                axis_map = {'X (depth)': 0, 'Y (height)': 1, 'Z (width)': 2}
                axis_idx = axis_map[slice_axis.value]
                slice_idx = slice_index.value
                
                # Extract slice
                if axis_idx == 0:
                    slice_2d = volume_to_show[slice_idx, :, :]
                elif axis_idx == 1:
                    slice_2d = volume_to_show[:, slice_idx, :]
                else:
                    slice_2d = volume_to_show[:, :, slice_idx]
                
                # Plot
                fig, ax = plt.subplots(figsize=(10, 10))
                im = ax.imshow(slice_2d, cmap='gray')
                ax.set_title(f'{title} - {slice_axis.value} Slice {slice_idx}', fontsize=14, fontweight='bold')
                ax.axis('off')
                plt.colorbar(im, ax=ax, label='Intensity')
                plt.tight_layout()
                plt.show()
                
                # Show slice statistics
                print(f"Slice Statistics:")
                print(f"  Shape: {slice_2d.shape}")
                print(f"  Min: {slice_2d.min():.2f}, Max: {slice_2d.max():.2f}")
                print(f"  Mean: {slice_2d.mean():.2f}, Std: {slice_2d.std():.2f}")
                if analyzer.segmented_volume is not None:
                    material_fraction = np.mean(slice_2d > 0)
                    print(f"  Material Fraction: {material_fraction:.2%}")
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Visualization complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error visualizing: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    segmentation_method.observe(update_segmentation_method, names='value')
    file_format_dropdown.observe(update_raw_params, names='value')
    slice_axis.observe(update_slice_range, names='value')
    load_button.on_click(load_volume_callback)
    segment_button.on_click(segment_volume_callback)
    compute_metrics_button.on_click(compute_metrics_callback)
    visualize_button.on_click(visualize_callback)
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


In [6]:
if WIDGETS_AVAILABLE:
    
    # Create RAW file parameters box (collapsible)
    raw_params_box = widgets.VBox([
        widgets.HTML("<b>RAW File Parameters:</b>"),
        HBox([dimensions_x, dimensions_y, dimensions_z]),
        dtype_dropdown
    ], layout=widgets.Layout(display='none'))
    
    def update_raw_params_display(change):
        """Show/hide RAW parameters"""
        if change['new'] == 'RAW':
            raw_params_box.layout.display = 'block'
        else:
            raw_params_box.layout.display = 'none'
    
    file_format_dropdown.observe(update_raw_params_display, names='value')
    
    # Create top panel with file loading
    top_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load XCT Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        raw_params_box,
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z,
            voxel_size_unit
        ]),
        normalize_check,
        HBox([load_button, volume_info_display])
    ])
    
    # Create segmentation panel
    segmentation_panel = widgets.VBox([
        widgets.HTML("<h3>🎯 Segmentation</h3>"),
        segmentation_method,
        manual_threshold,
        HBox([
            refine_segmentation_check,
            fill_holes_check
        ]),
        min_object_size,
        segment_button,
        segmentation_preview
    ])
    
    # Create metrics panel
    metrics_panel = widgets.VBox([
        widgets.HTML("<h3>📊 Metrics</h3>"),
        compute_metrics_button,
        metrics_display
    ])
    
    # Create visualization panel
    visualization_panel = widgets.VBox([
        widgets.HTML("<h3>🎨 Visualization</h3>"),
        HBox([
            slice_axis,
            slice_index
        ]),
        visualize_button,
        visualization_output
    ])
    
    # Create tabs for organized display
    results_tabs = Tab(children=[segmentation_panel, metrics_panel, visualization_panel])
    results_tabs.set_title(0, '🎯 Segmentation')
    results_tabs.set_title(1, '📊 Metrics')
    results_tabs.set_title(2, '🎨 Visualization')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>🔍 Interactive XCT Data Explorer</h1>"),
        top_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        results_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start exploring your XCT data.")
    print("\n💡 Tips:")
    print("   1. Enter file path and load volume")
    print("   2. Adjust segmentation parameters and segment")
    print("   3. Click 'Compute Metrics' to see results")
    print("   4. Use visualization tab to explore slices")
    print("   5. Adjust slice index and axis to explore different views")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start exploring your XCT data.

💡 Tips:
   1. Enter file path and load volume
   2. Adjust segmentation parameters and segment
   3. Click 'Compute Metrics' to see results
   4. Use visualization tab to explore slices
   5. Adjust slice index and axis to explore different views


In [7]:
# Example: Using XCTAnalyzer programmatically
print("📝 Example: Programmatic Analysis with XCTAnalyzer")
print("=" * 60)

# Initialize analyzer
example_analyzer = XCTAnalyzer(voxel_size=(0.1, 0.1, 0.1), target_unit='mm')
print(f"✅ Analyzer initialized with voxel_size={example_analyzer.voxel_size} mm")

# Example: Load volume (uncomment and provide path to test)
# example_analyzer.load_volume('path/to/your/data.dcm', normalize=True)
# print(f"✅ Volume loaded: shape={example_analyzer.volume.shape}")

# Example: Segment
# segmented = example_analyzer.segment(method='otsu', refine=True)
# print(f"✅ Volume segmented: shape={segmented.shape}")

# Example: Compute metrics
# metrics = example_analyzer.compute_metrics(include_surface_area=True)
# print(f"✅ Metrics computed:")
# print(f"   Volume: {metrics['volume']:.3f} mm³")
# print(f"   Void Fraction: {metrics['void_fraction']:.2%}")

print("\n💡 Uncomment the lines above and provide a file path to test programmatic analysis")
print("   Or use the interactive widgets above for a more user-friendly experience")


📝 Example: Programmatic Analysis with XCTAnalyzer
✅ Analyzer initialized with voxel_size=(0.1, 0.1, 0.1) mm

💡 Uncomment the lines above and provide a file path to test programmatic analysis
   Or use the interactive widgets above for a more user-friendly experience


## 7. Summary

### What We Learned

1. **Loading XCT Data**: 
   - Support for multiple formats (DICOM, TIFF, RAW, CSV, etc.)
   - Automatic format detection
   - Configurable voxel sizes and units

2. **Segmentation**:
   - Multiple methods (Otsu, Multi-threshold, Adaptive, Manual)
   - Morphological refinement options
   - Real-time preview

3. **Metrics Computation**:
   - Core metrics: volume, surface area, void fraction, relative density
   - Specific surface area
   - Material and void voxel counts

4. **Visualization**:
   - Interactive slice exploration
   - Multiple axis views
   - Real-time parameter adjustment

5. **XCTAnalyzer Class**:
   - Unified interface for all analysis operations
   - Programmatic access for automation
   - Integrated workflow management

### Next Steps

- **Notebook 02**: Preprocessing and Data Cleaning
  - Object filtering
  - Statistical fitting
  - Quality assessment

- **Notebook 03**: Core Analysis - Morphology and Porosity
  - Filament diameter estimation
  - Channel width analysis
  - Porosity distribution

- **Notebook 04**: Experimental Analysis
  - Flow analysis
  - Thermal analysis
  - Energy conversion

### Resources

- [Framework Documentation](../docs/README.md)
- [Module Reference](../docs/modules.md)
- [Tutorials](../docs/tutorials.md)
- [API Reference](../docs/api.md)
